In [2]:
import pandas as pd
import networkx as nx

In [204]:
D_data = pd.read_csv('data/Denmark/Merged.csv')
USA_data = pd.read_csv('data/USA/filtered_USA_votes.csv')

In [ ]:
#we check both ways, we calc both i -> j and j->i.
# person 221 as i and 1619 as j, and the other way. 1619 as i and 221 as j. is that a problem? 
#there are prob some variables that needs renaming

def calc_thres(PP, PA, PAV):
    party_agree = []
    for party in PP:
        print(party)
        memb_agree = []
        for member1 in PP.get(party):
            for member2 in set(PP.get(party)):
                if member1 != member2:
                    common = set(PAV.get(member1)) & set(PAV.get(member2)) # how many votes do they agree on 
                
                    common2 = set(PA.get(member1)) & set(PA.get(member2)) # how many same bills have they voted on

                    if len(common) != 0: # if they have at least one vote thats the same, (too avoid divide by 0)
                    
                        percentage = len(common) / len(common2)
                        memb_agree.append(percentage)
                    else:
                        memb_agree.append(0)
        if len(memb_agree) != 0:
            avgMemb = sum(memb_agree)/len(memb_agree)
            party_agree.append(avgMemb)
            print(avgMemb)
        else:
            print(party) # party where there is only one member
            print('1')
            #party_agree.append(1)
    avgParty = sum(party_agree)/len(party_agree)
    print(max(party_agree))
    return avgParty


In [ ]:
def edgelist_calc(PA, PAV, threshold):
    edgelist=[]
    for i in PAV: # person 1

        for j in PAV: # for loop for person 2

            if i != j: # checks they are not the same

                common = set(PAV.get(i)) & set(PAV.get(j)) # how many votes do they agree on 
                
                common2 = set(PA.get(i)) & set(PA.get(j)) # how many same bills have they voted on

                if len(common) != 0: # if they have at least one vote thats the same, (too avoid divide by 0)
                    
                    percentage = len(common) / len(common2)
                    if percentage > threshold:
                        edgelist.append((i,j))
                

    return list(set(edgelist))
    

In [ ]:
#----------------------------------------------------------------------------------------------------------------#
#----------------------------------------DENMARK-----------------------------------------------------------------#
#----------------------------------------------------------------------------------------------------------------#

In [201]:
#dict, key = id, values = (afstemningid, typeid)
# dict of all people with all their votes (voting id and what they voted)

PA = {}          #<<-------------- Person, Afstemning
for idx, row in D_data.iterrows():
    if row['aktørid'] not in PA:
        PA[row['aktørid']] = [(row['afstemningid'])]
    else:
        PA[row['aktørid']].append((row['afstemningid']))

PAV = {}          #<<-------------- Person, Afstemning, Vote
for idx, row in D_data.iterrows():
    if row['aktørid'] not in PAV:
        PAV[row['aktørid']] = [(row['afstemningid'],row['typeid_x'])]
    else:
        PAV[row['aktørid']].append((row['afstemningid'],row['typeid_x']))

""" 
PAVP = {}          #<<-------------- Person, Afstemning, Vote, Party
for idx, row in D_data.iterrows():
    if row['aktørid'] not in PAVP:
        PAVP[row['aktørid']] = [(row['afstemningid'],row['typeid_x'],row['party'],row['navn'])]
    else:
        PAVP[row['aktørid']].append((row['afstemningid'],row['typeid_x'],row['party'],row['navn']))
 """
#dict of parties with memberid as values

PP = {}          #<<-------------- Party, Person
for idx, row in D_data.iterrows():
    if row['party'] not in PP:
        PP[row['party']] = [(row['aktørid'])]
    elif row['aktørid'] not in PP.get(row['party']):
        PP[row['party']].append((row['aktørid']))


In [ ]:
D_thres = calc_thres(PP,PA,PAV)
D_thres

In [196]:
edgelist = edgelist_calc(PA,PAV, res)
G_denmark = nx.from_edgelist(edgelist)

In [183]:
len(edgelist)

76572

In [185]:
list_of_dangle = [node for node in G_denmark.nodes if G_denmark.degree(node) == 0]
list_of_dangle

[]

In [ ]:
#----------------------------------------------------------------------------------------------------------------#
#------------------------------------------USA-------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------------------------#


In [4]:
U_PA = {}          #<<-------------- Person, Afstemning
for idx, row in USA_data.iterrows():
    if row['congress'] == 118:
        if row['icpsr'] not in U_PA:
            U_PA[row['icpsr']] = [(row['rollnumber'])]
        else:
            U_PA[row['icpsr']].append((row['rollnumber']))

U_PAV = {}          #<<-------------- Person, Afstemning, Vote
for idx, row in USA_data.iterrows():
    if row['congress'] == 118:
        if row['icpsr'] not in U_PAV:
            U_PAV[row['icpsr']] = [(row['rollnumber'],row['cast_code'])]
        else:
            U_PAV[row['icpsr']].append((row['rollnumber'],row['cast_code']))

""" 
U_PAVP = {}          #<<-------------- Person, Afstemning, Vote, Party
for idx, row in USA_data.iterrows():
    if row['congress'] == 118:
        if row['icpsr'] not in U_PAVP:
            U_PAVP[row['icpsr']] = [(row['rollnumber'],row['cast_code'],row['party_code'])]
        else:
            U_PAVP[row['icpsr']].append((row['rollnumber'],row['cast_code'],row['party_code']))
 """
#dict of parties with memberid as values

U_PP = {}          #<<-------------- Party, Person
for idx, row in USA_data.iterrows():
    if row['congress'] == 118:
        if row['party_code'] not in U_PP:
            U_PP[row['party_code']] = [(row['icpsr'])]
        elif row['icpsr'] not in U_PP.get(row['party_code']):
            U_PP[row['party_code']].append((row['icpsr']))

In [7]:
U_thres = calc_thres(U_PP, U_PA, U_PAV)

200
0.8710690231164628
100
0.9435322225344877
0.9435322225344877


In [8]:
U_edgelist = edgelist_calc(U_PA, U_PAV, U_thres)
G_USA = nx.from_edgelist(U_edgelist)

In [10]:
list_of_dangle = [node for node in G_USA.nodes if G_USA.degree(node) == 0]
list_of_dangle

[]